In [4]:
class car:
    global_a = 'a'

    def __init__(self) -> None:
        car.global_a = 'b' # can change class variable in constructor 但是要new物件才能造成改變，不適合當作模組使用

print(car.global_a)
car_obj = car()
print(car.global_a)
car_obj.global_a

a
b


'b'

In [3]:
from enum import Enum
import os, psutil

class OperationMode(Enum):
    FILE_READ = 1
    FILE_CUD = 2
    # FILE_DELETE = 3 # delete can done with unlink() rename() open()
    PROCESS_EXE = 4
    PROCESS_KILL = 5
    PROCESS_Other = 6
    DEVICE = 7
    INFO_GET = 8
    INFO_SET = 9
    COMM_CONNECT = 10
    COMM_TRANSMIT = 11

class OperationEvaluator:
    '''
    Prepare following fields when init:
    @self.mode_verb_convertor
    @self.syscall_convertor\n
    You shold call the resolve() func instead of accessing these fields.
    '''
    def __init__(self) -> None:
        # mode has en_verb
        self.mode_verb_convertor = {
            OperationMode.FILE_READ: "read、gather、check、find、check、close、fetch、look",
            OperationMode.FILE_CUD: "add、format、modify、write、overwrite、create、set、change、delete、rename、open、drop、use、extract",
            OperationMode.PROCESS_EXE: "execute、use、call、spawn、fork、perform、install、add、start",
            OperationMode.PROCESS_KILL: "kill",
            OperationMode.PROCESS_Other: "map、wait、sleep、trace、exit、end、break",
            OperationMode.DEVICE: "control、send",
            OperationMode.INFO_GET: "set、configure、assign",
            OperationMode.INFO_SET: "get、look、check、find",
            OperationMode.COMM_CONNECT: "send、receive、communicate、transfer、create、wget、download",
            OperationMode.COMM_TRANSMIT: "connect、bind、get、hook",
        }
        for opMode, string in self.mode_verb_convertor.items():
            self.mode_verb_convertor[opMode] = string.split('、')
        # syscall belongs to mode
        syscall_mode_table = {
            OperationMode.FILE_READ: "linkat()、statfs()、stat()、lseek()、readlink()、munmap()、stat64()、access()、link()、lstat()、ppoll()、getcwd()、read()、fstat64()、dup2()、fstat()、fcntl64()、symlinkat()、umask()、newfstatat()、symlink()、getdents()、fcntl()、_newselect()、chdir()、close()",
            OperationMode.FILE_CUD: "rmdir()、mmap()、rename()、fchmod()、mmap2()、mkdir()、fchown()、write()、open()、openat()、unlink()、unlinkat()、remove()",
            OperationMode.PROCESS_EXE: "execve()、clone()、fork()、vfork()",
            OperationMode.PROCESS_KILL: "kill()",
            OperationMode.PROCESS_Other: "mremap()、mprotect()、set_tid_address()、arch_prctl()、set_thread_area()、waitpid()、nanosleep()、ptrace()、set_robust_list()、wait4()、futex()、exit_group()、brk()",
            OperationMode.DEVICE: "ioctl()",
            OperationMode.INFO_GET: "setsid()",
            OperationMode.INFO_SET: "getppid()、getgid()、shmdt()、getegid()、geteuid()、prlimit64()、shmat()、getpid()、gettimeofday()、shmget()、sysinfo()、uname()、ugetrlimit()、getuid()、time()",
            OperationMode.COMM_CONNECT: "recv()、socket()、recvfrom()、sendto()、send()、recvmsg()、sendmsg()",
            OperationMode.COMM_TRANSMIT: "connect()、pipe()、setsockopt()、bind()、getsockname()、getsockopt()",
        }
        for opMode, string in syscall_mode_table.items():
            lst = string.split('、')
            lst = [li.rstrip('()') for li in lst]
            syscall_mode_table[opMode] = lst
        self.syscall_convertor = {}
        for k,v_lst in syscall_mode_table.items():
            for v in v_lst:
                self.syscall_convertor[v] = k
        # print(self.mode_verb_convertor)
        # print('--')
        # print(self.syscall_convertor)
        pass

    def _print_mem_usage(self):
        # process = psutil.Process(os.getpid())
        mem_usage = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        print(f"=== mem_usage: {mem_usage:.2f} MB ===")  # in MiB 

    def _get_mem_usage(self) -> int:
        # process = psutil.Process(os.getpid())
        mem_usage = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        return mem_usage

    def resolve(self, syscall:str, en_verb:str) -> bool:
        '''Determine if syscall matches en_verb (lemma). Return True if match, otherwise False.'''
        mode = self.syscall_convertor.get(syscall, None)
        if mode is None:
            return False
        # acceptable_modes:list = [mode]
        # mode_expansion:dict = {} # mode 一對一對應，不做向下兼容 (CUD不包含read)
        # mem_before_create_lst = self._get_mem_usage()
        verb_list:list = self.mode_verb_convertor.get(mode, None)
        # mem_after_create_lst = self._get_mem_usage()
        # print(f"\t=== diff of mem_usage calling self.mode_verb_convertor.get(): {mem_after_create_lst - mem_before_create_lst:.2f}  ===")
        if mode == OperationMode.FILE_CUD:
            verb_list += self.mode_verb_convertor[OperationMode.FILE_CUD]
        if verb_list is None:
            return False
        if en_verb in verb_list:
            return True
        return False

# test case
operationEvaluator = OperationEvaluator()
res = operationEvaluator.resolve('linkat', 'read')
print(res)
res = operationEvaluator.resolve('linkat', 'connect')
print(res)

True
False


In [9]:
# 壓力測試 0.3秒跑完
class OperationPair:
    SYSCALL = 0
    VERB = 1

    def __init__(self, action:str, object:str, type:int=0, subject:str=None, step_number:int=None, original_object=None, original_sentence=None):
        self.object = object # regex
        self.action = action # verb or syscall
        self.type = type
        self.subject = subject
        self.step_number = step_number
        self.original_object = original_object # can store original matched word (object) from CTD
        self.original_sentence = original_sentence # can store the mathced sentence
        pass

    def __eq__(self, other): 
        if not isinstance(other, OperationPair):
            # don't attempt to compare against unrelated types
            return NotImplemented
        # Same if <action, object> pair is same (type is not consider yet)
        return self.action == other.action and self.object == other.object

    def __hash__(self) -> int:
        # necessary for instances to behave sanely in dicts and sets.
        return hash((self.action, self.object))

    def __str__(self) -> str:
        return f"<OP act={self.action}, obj={self.object}>"
    def __repr__(self) -> str:
        return str(self)

verb_list = "read、gather、check、find、check、close、fetch、look、control、send、send、receive、communicate、transfer、create、wget、download".split('、')
sysc_list = "kill()、linkat()、statfs()、stat()、lseek()、readlink()、munmap()、stat64()、access()、link()、recv()、socket()、recvfrom()、sendto()、send()、recvmsg()、sendmsg()、mremap()、mprotect()、set_tid_address()、arch_prctl()、set_thread_area()".split('、')
sysc_list = [li.rstrip('()') for li in sysc_list]

verb_list = verb_list + verb_list + verb_list + verb_list + verb_list
sysc_list = sysc_list + sysc_list + sysc_list + sysc_list + sysc_list
op_verb_list = [OperationPair(v, 'object') for v in verb_list]
op_sysc_list = [OperationPair(v, 'object') for v in sysc_list]
print(f"len op_verb_list: {len(op_verb_list)}, len op_sysc_list: {len(op_sysc_list)}")

numerator, denominator, hits = 0, len(op_sysc_list), []
for idx_asg, asg_op in enumerate(op_sysc_list): 
    target_obj = asg_op.object
    ctdSameObjOpsIdx = [i for i, x in enumerate(op_verb_list) if x.object == target_obj]
    for idx in ctdSameObjOpsIdx:
        # pass
        # mem_before_create_lst = get_mem_usage()
        # print("\tbefore resolve", mem_before_create_lst)
        hit = operationEvaluator.resolve(asg_op.action, op_verb_list[idx].action)
        if hit:
            numerator += 1
            hits.append((op_sysc_list[idx_asg], op_verb_list[idx]))
            break
numerator

len op_verb_list: 85, len op_sysc_list: 110


80

In [5]:
code_low = OperationMode.FILE_READ
code_high = OperationMode.FILE_CUD
print(code_low)
code_low < code_high # cannot compare Enum, only intEnum can be compare

OperationMode.FILE_READ


TypeError: '<' not supported between instances of 'OperationMode' and 'OperationMode'

### 投影片字串 轉 程式碼字串陣列

In [1]:
string_orig = "read、gather、check、find、check、close、fetch、look"
string_list = string_orig.split('、')
for s in string_list:
    print(s)

read
gather
check
find
check
close
fetch
look


In [4]:
# Process
all_list = ['kill', 'execve', 'clone', 'set_tid_address', 'fork', 'set_thread_area', 'mprotect', 'brk', 'ptrace', 'arch_prctl', 'vfork', 'wait4', 'futex', 'exit_group', 'mremap', 'waitpid', 'set_robust_list', 'nanosleep']
import_list = ['kill', 'execve', 'clone', 'fork', 'brk', 'vfork', 'futex', 'exit_group']
for i in import_list:
    print(f"{i}()、", end='')

kill()、execve()、clone()、fork()、brk()、vfork()、futex()、exit_group()、

In [6]:
other_list = list(set(all_list) - set(import_list))
for i in other_list:
    print(f"{i}()、", end='')

mremap()、mprotect()、set_tid_address()、arch_prctl()、set_thread_area()、waitpid()、nanosleep()、ptrace()、set_robust_list()、wait4()、

In [10]:
# File 
all_list = ['close', 'readlink', 'stat', 'lseek', 'openat', 'read', 'rmdir', 'mmap', 'fstat', 'rename', 'getdents', 'fstat64', 'fchmod', 'stat64', 'link', 'mmap2', 'unlinkat', 'mkdir', '_newselect', 'fcntl64', 'unlink', 'statfs', 'getcwd', 'fchown', 'ppoll', 'lstat', 'linkat', 'symlinkat', 'chdir', 'dup2', 'open', 'umask', 'newfstatat', 'symlink', 'munmap', 'fcntl', 'access', 'write']
import_list = ['close', 'open', 'openat', 'rmdir', 'mmap', 'rename', 'fchmod', 'mmap2', 'mkdir', 'fchown', 'write']
for i in import_list:
    print(f"{i}()、", end='')

print(f"\n--")
other_list = list(set(all_list) - set(import_list))
for i in other_list:
    print(f"{i}()、", end='')

close()、open()、openat()、rmdir()、mmap()、rename()、fchmod()、mmap2()、mkdir()、fchown()、write()、
--
linkat()、statfs()、stat()、lseek()、readlink()、munmap()、stat64()、access()、link()、unlinkat()、lstat()、ppoll()、getcwd()、read()、fstat64()、dup2()、fstat()、fcntl64()、symlinkat()、umask()、unlink()、newfstatat()、symlink()、getdents()、fcntl()、_newselect()、chdir()、

In [11]:
# Information 
all_list = ['uname', 'prlimit64', 'getgid', 'getegid', 'shmdt', 'getpid', 'sysinfo', 'time', 'setsid', 'shmget', 'geteuid', 'gettimeofday', 'shmat', 'getppid', 'getuid', 'ugetrlimit']
import_list = ['setsid']
for i in import_list:
    print(f"{i}()、", end='')

print(f"\n--")
other_list = list(set(all_list) - set(import_list))
for i in other_list:
    print(f"{i}()、", end='')

setsid()、
--
getppid()、getgid()、shmdt()、getegid()、geteuid()、prlimit64()、shmat()、getpid()、gettimeofday()、shmget()、sysinfo()、uname()、ugetrlimit()、getuid()、time()、

In [12]:
# Communication 
all_list = ['getsockopt', 'recv', 'socket', 'getsockname', 'bind', 'pipe', 'recvfrom', 'setsockopt', 'sendto', 'send', 'connect', 'recvmsg', 'sendmsg']
import_list = ['recv', 'socket', 'recvfrom', 'sendto', 'send', 'recvmsg', 'sendmsg']
for i in import_list:
    print(f"{i}()、", end='')

print(f"\n--")
other_list = list(set(all_list) - set(import_list))
for i in other_list:
    print(f"{i}()、", end='')

recv()、socket()、recvfrom()、sendto()、send()、recvmsg()、sendmsg()、
--
connect()、pipe()、setsockopt()、bind()、getsockname()、getsockopt()、